In [ ]:
# === Recreate val split, run inference, compute per-class metrics ===
# Edit these:
DATA_FILE = "C:/Users/chris/Desktop/Documents/Code/MultimodalDataChallenge2025/metadata.csv"
IMAGE_DIR = "A:/FungiImages/FungiImages" # root containing fungi_train/*
CKPT_DIR  = "C:/Users/chris/Desktop/Documents/Code/MultimodalDataChallenge2025/checkpoints/" # where best_accuracy.pth lives

import os, pandas as pd, numpy as np
from pathlib import Path
import torch, torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from albumentations import Compose, Normalize, Resize
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from torchvision import models
from PIL import Image

# --- Transforms (valid) ---
def get_valid_tf():
    w, h = 224, 224
    return Compose([Resize(w, h),
                    Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
                    ToTensorV2()])

# --- Dataset ---
class FungiDataset(Dataset):
    def __init__(self, df, root, transform):
        self.df = df.reset_index(drop=True)
        self.root, self.t = root, transform
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        fp = self.df["filename_index"].iloc[i].strip()
        y  = int(self.df["taxonID_index"].iloc[i])  # <- restore label extraction
        path = os.path.join(self.root, fp)
        #print(f"[DBG] idx={i} -> {path}")

        if not os.path.exists(path):
            raise FileNotFoundError(f"Image not found: {path}. Check IMAGE_DIR.")

        if self.t is None:
            raise RuntimeError("Transform is None (get_valid_tf() not applied).")

        try:
            with Image.open(path) as img:
                im = np.array(img.convert("RGB"))
            im = self.t(image=im)["image"]
            return im, y, fp
        except Exception as e:
            raise RuntimeError(f"__getitem__ failed for {path}: {type(e).__name__}: {e}")


# --- Recreate EXACT split ---
df = pd.read_csv(DATA_FILE)
df["filename_index"] = df["filename_index"].astype(str).str.strip()
train_df_all = df[df["filename_index"].str.startswith("fungi_train")].dropna(subset=["taxonID_index"]).copy()
train_df_all["taxonID_index"] = train_df_all["taxonID_index"].astype(int)

train_df, val_df = train_test_split(train_df_all, test_size=0.2, random_state=42)

# --- Model ---
num_classes = train_df_all["taxonID_index"].nunique()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.classifier[1].in_features, num_classes)
)
state_path = os.path.join(CKPT_DIR, "best_accuracy.pth")
model.load_state_dict(torch.load(state_path, map_location=device))
model.to(device).eval()

# --- Inference on VAL ---
val_ds = FungiDataset(val_df, IMAGE_DIR, transform=get_valid_tf())
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=0)


c:\Users\chris\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\chris\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# does a single sample load?
x0, y0, f0 = val_ds[0]; print(f0, x0.shape)

# does one batch load?
tmp_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=0)
xb, yb, fb = next(iter(tmp_loader)); print(xb.shape, len(fb))

[DBG] idx=0 -> A:/FungiImages/FungiImages\fungi_train004036.jpg
fungi_train004036.jpg torch.Size([3, 224, 224])
[DBG] idx=0 -> A:/FungiImages/FungiImages\fungi_train004036.jpg
[DBG] idx=1 -> A:/FungiImages/FungiImages\fungi_train024284.jpg
[DBG] idx=2 -> A:/FungiImages/FungiImages\fungi_train005038.jpg
[DBG] idx=3 -> A:/FungiImages/FungiImages\fungi_train001087.jpg
torch.Size([4, 3, 224, 224]) 4


In [ ]:
from tqdm import tqdm

y_true, y_pred, names = [], [], []
with torch.no_grad():
    for x, y, fn in tqdm(val_loader, desc="Validating", unit="batch"):
        x = x.to(device)
        logits = model(x)
        preds = logits.argmax(1).cpu().numpy()
        y_pred.extend(preds.tolist())
        y_true.extend(y.numpy().tolist())
        names.extend(fn)

y_true = np.array(y_true); y_pred = np.array(y_pred)

# --- Metrics ---
acc = accuracy_score(y_true, y_pred)
rep = classification_report(y_true, y_pred, output_dict=True, zero_division=0)
rep_df = pd.DataFrame(rep).transpose()
per_class = rep_df[[i for i in rep_df.index if i.isdigit()]].T
per_class.index = per_class.index.astype(int)
per_class = per_class.sort_index().rename(columns={"precision":"prec","recall":"rec","f1-score":"f1","support":"n"})

labels_sorted = sorted(per_class.index.unique())
cm = confusion_matrix(y_true, y_pred, labels=labels_sorted)

# Misclassifications list
mis = pd.DataFrame({
    "filename_index": names,
    "y_true": y_true,
    "y_pred": y_pred
})
mis = mis[mis["y_true"] != mis["y_pred"]]

# --- Save ---
out = Path(CKPT_DIR)
out.mkdir(parents=True, exist_ok=True)
per_class.to_csv(out/"val_per_class_metrics.csv", index_label="class_index")
pd.DataFrame(cm, index=labels_sorted, columns=labels_sorted).to_csv(out/"val_confusion_matrix.csv")
rep_df.loc[["accuracy","macro avg","weighted avg"]].to_csv(out/"val_overall_metrics.csv")
mis.to_csv(out/"val_misclassified.csv", index=False)

print(f"VAL accuracy: {acc:.4f}")
print(f"Saved: {out/'val_per_class_metrics.csv'}, {out/'val_confusion_matrix.csv'}, {out/'val_overall_metrics.csv'}, {out/'val_misclassified.csv'}")

Validating:   0%|          | 0/162 [00:00<?, ?batch/s]

[DBG] idx=0 -> A:/FungiImages/FungiImages\fungi_train004036.jpg
[DBG] idx=1 -> A:/FungiImages/FungiImages\fungi_train024284.jpg
[DBG] idx=2 -> A:/FungiImages/FungiImages\fungi_train005038.jpg
[DBG] idx=3 -> A:/FungiImages/FungiImages\fungi_train001087.jpg
[DBG] idx=4 -> A:/FungiImages/FungiImages\fungi_train015219.jpg
[DBG] idx=5 -> A:/FungiImages/FungiImages\fungi_train011541.jpg
[DBG] idx=6 -> A:/FungiImages/FungiImages\fungi_train002435.jpg
[DBG] idx=7 -> A:/FungiImages/FungiImages\fungi_train018797.jpg
[DBG] idx=8 -> A:/FungiImages/FungiImages\fungi_train022151.jpg
[DBG] idx=9 -> A:/FungiImages/FungiImages\fungi_train005050.jpg
[DBG] idx=10 -> A:/FungiImages/FungiImages\fungi_train021407.jpg
[DBG] idx=11 -> A:/FungiImages/FungiImages\fungi_train013093.jpg
[DBG] idx=12 -> A:/FungiImages/FungiImages\fungi_train015736.jpg
[DBG] idx=13 -> A:/FungiImages/FungiImages\fungi_train025748.jpg
[DBG] idx=14 -> A:/FungiImages/FungiImages\fungi_train016382.jpg
[DBG] idx=15 -> A:/FungiImages/Fung

Validating:   1%|          | 1/162 [00:13<36:40, 13.67s/batch]

[DBG] idx=32 -> A:/FungiImages/FungiImages\fungi_train025253.jpg
[DBG] idx=33 -> A:/FungiImages/FungiImages\fungi_train012310.jpg
[DBG] idx=34 -> A:/FungiImages/FungiImages\fungi_train005937.jpg
[DBG] idx=35 -> A:/FungiImages/FungiImages\fungi_train001972.jpg
[DBG] idx=36 -> A:/FungiImages/FungiImages\fungi_train015259.jpg
[DBG] idx=37 -> A:/FungiImages/FungiImages\fungi_train005130.jpg
[DBG] idx=38 -> A:/FungiImages/FungiImages\fungi_train005683.jpg
[DBG] idx=39 -> A:/FungiImages/FungiImages\fungi_train011191.jpg
[DBG] idx=40 -> A:/FungiImages/FungiImages\fungi_train001684.jpg
[DBG] idx=41 -> A:/FungiImages/FungiImages\fungi_train008919.jpg
[DBG] idx=42 -> A:/FungiImages/FungiImages\fungi_train013139.jpg
[DBG] idx=43 -> A:/FungiImages/FungiImages\fungi_train014802.jpg
[DBG] idx=44 -> A:/FungiImages/FungiImages\fungi_train018911.jpg
[DBG] idx=45 -> A:/FungiImages/FungiImages\fungi_train004706.jpg
[DBG] idx=46 -> A:/FungiImages/FungiImages\fungi_train000382.jpg
[DBG] idx=47 -> A:/FungiI

Validating:   1%|          | 2/162 [00:31<42:22, 15.89s/batch]

...

[DBG] idx=608 -> A:/FungiImages/FungiImages\fungi_train015532.jpg
[DBG] idx=609 -> A:/FungiImages/FungiImages\fungi_train009172.jpg
[DBG] idx=610 -> A:/FungiImages/FungiImages\fungi_train013413.jpg
[DBG] idx=611 -> A:/FungiImages/FungiImages\fungi_train022595.jpg
[DBG] idx=612 -> A:/FungiImages/FungiImages\fungi_train001346.jpg
[DBG] idx=613 -> A:/FungiImages/FungiImages\fungi_train021184.jpg
[DBG] idx=614 -> A:/FungiImages/FungiImages\fungi_train018153.jpg
[DBG] idx=615 -> A:/FungiImages/FungiImages\fungi_train014811.jpg
[DBG] idx=616 -> A:/FungiImages/FungiImages\fungi_train020461.jpg
[DBG] idx=617 -> A:/FungiImages/FungiImages\fungi_train014280.jpg
[DBG] idx=618 -> A:/FungiImages/FungiImages\fungi_train022541.jpg
[DBG] idx=619 -> A:/FungiImages/FungiImages\fungi_train024331.jpg
[DBG] idx=620 -> A:/FungiImages/FungiImages\fungi_train022913.jpg
[DBG] idx=621 -> A:/FungiImages/FungiImages\fungi_train014875.jpg
[DBG] idx=622 -> A:/FungiImages/FungiImages\fungi_train017001.jpg
[DBG] idx=

Validating:  12%|█▏        | 20/162 [03:01<17:55,  7.57s/batch]

[DBG] idx=640 -> A:/FungiImages/FungiImages\fungi_train021236.jpg
[DBG] idx=641 -> A:/FungiImages/FungiImages\fungi_train014488.jpg
[DBG] idx=642 -> A:/FungiImages/FungiImages\fungi_train016775.jpg
[DBG] idx=643 -> A:/FungiImages/FungiImages\fungi_train025494.jpg
[DBG] idx=644 -> A:/FungiImages/FungiImages\fungi_train010080.jpg
[DBG] idx=645 -> A:/FungiImages/FungiImages\fungi_train004689.jpg
[DBG] idx=646 -> A:/FungiImages/FungiImages\fungi_train024821.jpg
[DBG] idx=647 -> A:/FungiImages/FungiImages\fungi_train019543.jpg
[DBG] idx=648 -> A:/FungiImages/FungiImages\fungi_train004119.jpg
[DBG] idx=649 -> A:/FungiImages/FungiImages\fungi_train018835.jpg
[DBG] idx=650 -> A:/FungiImages/FungiImages\fungi_train004740.jpg
[DBG] idx=651 -> A:/FungiImages/FungiImages\fungi_train005504.jpg
[DBG] idx=652 -> A:/FungiImages/FungiImages\fungi_train015239.jpg
[DBG] idx=653 -> A:/FungiImages/FungiImages\fungi_train014204.jpg
[DBG] idx=654 -> A:/FungiImages/FungiImages\fungi_train013099.jpg
[DBG] idx=

Validating:  13%|█▎        | 21/162 [03:09<18:27,  7.85s/batch]

[DBG] idx=672 -> A:/FungiImages/FungiImages\fungi_train014396.jpg
[DBG] idx=673 -> A:/FungiImages/FungiImages\fungi_train015403.jpg
[DBG] idx=674 -> A:/FungiImages/FungiImages\fungi_train013898.jpg
[DBG] idx=675 -> A:/FungiImages/FungiImages\fungi_train008423.jpg
[DBG] idx=676 -> A:/FungiImages/FungiImages\fungi_train002434.jpg
[DBG] idx=677 -> A:/FungiImages/FungiImages\fungi_train019270.jpg
[DBG] idx=678 -> A:/FungiImages/FungiImages\fungi_train000733.jpg
[DBG] idx=679 -> A:/FungiImages/FungiImages\fungi_train014568.jpg
[DBG] idx=680 -> A:/FungiImages/FungiImages\fungi_train018672.jpg
[DBG] idx=681 -> A:/FungiImages/FungiImages\fungi_train013814.jpg
[DBG] idx=682 -> A:/FungiImages/FungiImages\fungi_train002222.jpg
[DBG] idx=683 -> A:/FungiImages/FungiImages\fungi_train018927.jpg
[DBG] idx=684 -> A:/FungiImages/FungiImages\fungi_train001103.jpg
[DBG] idx=685 -> A:/FungiImages/FungiImages\fungi_train000150.jpg
[DBG] idx=686 -> A:/FungiImages/FungiImages\fungi_train004160.jpg
[DBG] idx=